**Calcutate BLEU & ROUGE metrics for our LFQA**


In [ ]:
!pip install transformers datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

model_name = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer2 = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

In [ ]:
def generate_answers(question, contenxt):
  generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
  tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)

In [ ]:
from datasets import load_metric

# تحميل المقاييس
rouge_metric = load_metric("rouge")
bleu_metric = load_metric("bleu")

# حساب ROUGE
rouge_scores = rouge_metric.compute(predictions=generated_answers, references=test_answers, use_stemmer=True)
print("ROUGE Scores:", rouge_scores)

# إعداد الإجابات بالشكل المطلوب لـ BLEU
def prepare_bleu_references(references):
    return [[ref.split()] for ref in references]

def prepare_bleu_predictions(predictions):
    return [pred.split() for pred in predictions]

bleu_references = prepare_bleu_references(test_answers)
bleu_predictions = prepare_bleu_predictions(generated_answers)

# حساب BLEU
bleu_score = bleu_metric.compute(predictions=bleu_predictions, references=bleu_references)
print("BLEU Score:", bleu_score)


In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from datasets import load_dataset
dataset = load_dataset('vblagoje/lfqa', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/226147 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3020 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 10000
})

In [ ]:
import json
import random
from datasets import load_dataset, Dataset, DatasetDict

# تحميل مجموعة البيانات
dataset = load_dataset('vblagoje/lfqa')
data = dataset['test']

# تحديد الفئات الفرعية
subreddits = ['explainlikeimfive', 'AskHistorians', 'askscience']

# وظيفة لتصفية السجلات حسب الفئة الفرعية
def filter_by_subreddit(data, subreddit):
    return [record for record in data if record['subreddit'] == subreddit]

# تخزين السجلات المختارة
sampled_records = []

# تصفية السجلات واختيار عشوائي لـ 100 سجل من كل فئة فرعية
for subreddit in subreddits:
    filtered_data = filter_by_subreddit(data, subreddit)
    if len(filtered_data) < 100:
        raise ValueError(f"الفئة الفرعية '{subreddit}' تحتوي على أقل من 100 سجل.")
    sampled_records.extend(random.sample(filtered_data, 100))

# إنشاء مجموعة بيانات جديدة باستخدام Dataset.from_dict
sampled_train_dataset = Dataset.from_list(sampled_records)

# حفظ مجموعة البيانات الجديدة إلى ملف JSON
with open('test.json', 'w') as f:
    json.dump(sampled_records, f, indent=4)  # حفظ كقائمة من القواميس


In [ ]:
sampled_train_dataset

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 300
})

In [ ]:
!pip install accelerate tqdm

In [ ]:
support_doc = load_dataset('vblagoje/lfqa_support_docs')
lfqa = load_dataset('fahdsoliman/lfqa_dataset')

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
support_doc

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output', 'meta'],
        num_rows: 226147
    })
    validation: Dataset({
        features: ['id', 'input', 'output', 'meta'],
        num_rows: 3020
    })
})

In [ ]:
import math
import re

In [ ]:
def cleanup_references(text):
    # URL reference where we need to remove both the link text and URL
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal
    # views on slavery ([1](_URL_2_ & pg=PA173), [2](_URL_1_), [3](_URL_5_)).
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal views on slavery.
    result = re.sub(r"[\(\s]*\[\d+\]\([^)]+\)[,)]*", "", text, 0, re.MULTILINE)

    # URL reference where we need to preserve link text but remove URL
    # At the outbreak of the Civil War, [Leyburn left his church](_URL_19_) and joined the South.
    # At the outbreak of the Civil War, Leyburn left his church and joined the South.
    result = re.sub(r"\[([^]]+)\]\([^)]+\)", "\\1", result, 0, re.MULTILINE)

    # lastly remove just dangling _URL_[0-9]_ URL references
    result = re.sub(r"_URL_\d_", "", result, 0, re.MULTILINE)
    return result

In [ ]:
def clean_answer(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = re.sub(r"BULLET::::-", "", result)
    return result.strip()


def clean_question(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = result.replace("[deleted]", "")
    return result.lower().strip()


def prepare_support_docs(example):
    provenances = example["output"][-1]["provenance"]
    context = "<P> " + " <P> ".join([p["text"] for p in provenances])
    return {"context": context}

In [ ]:
def preprocess_eli5(examples, **fn_kwargs):
    document_cache = fn_kwargs["document_cache"]
    training = fn_kwargs.get("training", True)
    extra_answer_threshold = fn_kwargs.get("extra_answer_threshold", 10)
    include_selftext = fn_kwargs.get("include_selftext", False)
    exclude_answer_patterns = fn_kwargs.get("exclude_answer_patterns", [])

    questions, contexts, answers = [], [], []
    for q_id, question, selftext, answer in zip(examples["q_id"], examples["title"], examples["selftext"],
                                                examples["answers"]):
        accepted_answer_idx = []
        if training:
            accepted_answer_idx = [idx for idx, score in enumerate(answer["score"]) if
                                   score > extra_answer_threshold]
        if not training or not accepted_answer_idx:
            accepted_answer_idx = [0]
        document = document_cache[q_id]
        for idx in accepted_answer_idx:
            skip_answer = any([p.search(answer["text"][idx]) for p in exclude_answer_patterns])
            if skip_answer:
                continue
            if include_selftext:
                questions.append(clean_question(f"{question} {selftext}"))
            else:
                questions.append(clean_question(question))
            contexts.append(document.lower().strip())
            answers.append(clean_answer(answer["text"][idx]))

    return {"question": questions, "context": contexts, "answer": answers}

In [ ]:
from tqdm.auto import tqdm
from accelerate import Accelerator
accelerator = Accelerator()

processed_datasets = {}
support_docs_prepared = {}
with accelerator.main_process_first():
    support_docs_prepared = support_doc["test"].map(prepare_support_docs,
                                                            batched=False,
                                                            desc="Preparing support docs",
                                                            )
    column_names = lfqa["test"].column_names
    d_cache = dict([(e["id"], e["context"]) for e in tqdm(support_docs_prepared,
                                                          desc=f"Adding support docs to LFQA ")])
    processed_datasets["test"] = lfqa["test"].map(preprocess_eli5,
                                                batched=True,
                                                remove_columns=column_names,
                                                desc="Preparing dataset for tokenization",
                                                fn_kwargs={"document_cache": d_cache,
                                                            "exclude_answer_patterns": [re.compile("not sure what you"),
                                                                                        re.compile("\n\n >")]}
                                                )


KeyError: 'test'